## Поиск похожих по эмбеддингам

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [26]:
!pip install pymorphy2
!pip install stop_words
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
# Импорт библиотек

import pandas as pd
import numpy as np
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

1. Объединить в одну выборку

In [28]:
# Загрузка данных

df1 = pd.read_csv('positive.csv', sep=';', header = None, usecols = [3], names=['tweet'], encoding='UTF-8')
df2 = pd.read_csv('negative.csv', sep=';', header = None, usecols = [3], names=['tweet'], encoding='UTF-8')

In [29]:
# Объединение двух датафреймов

df = df1.append(df2, ignore_index = True, sort = False)

In [30]:
df.head(5)

,tweet
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


2. На основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

In [31]:
# Препроцессинг текста

def preprocess_txt(line):
    line = re.sub(r'RT*', '', line)
    line = re.sub(r'@[\w]*', ' ', line)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [32]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

for line in df['tweet']:
    spls = preprocess_txt(line)
    sentences.append(spls)

In [33]:
sentences = [i for i in sentences if len(i) > 2]

In [34]:
# Создание модели Word2Vec

modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=3)

In [35]:
# Создание модели FastText

modelFT = FastText(sentences=sentences, size=300, min_count=3, window=5, workers=8)

In [36]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

counter = 0
index_map = {}

for line in df['tweet']:
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line
    tweet = preprocess_txt(line)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in tweet:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [37]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

3. Проверить насколько хорошо работают подходы

In [38]:
TEXT = "сегодня хорошая погода"

In [39]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['хороший день..такая погода приятная)снежочек..только скользко блеать.',
 'Погода прекрасна, настрій піднесений, життя чудове!;)',
 'RT @MissUFO: Прогноз погоды для ИТ на ближайшее время ) http://t.co/hj6x0KyK8Y',
 'RT @MissUFO: Прогноз погоды для ИТ на ближайшее время ) http://t.co/hj6x0KyK8Y',
 'RT @MissUFO: Прогноз погоды для ИТ на ближайшее время ) http://t.co/hj6x0KyK8Y']

In [40]:
get_response(TEXT, ft_index, modelFT, index_map)

['На улице конечно хорошо,но лучше бы было солнце еще:)',
 'Самый лучший день ! ))) Первый хороший снег нападал)))))',
 '@agalitsky солнечно! :) привезите в москву кусочек хорошей погоды, хотя сегодня и здесь солнце весь день!',
 'Хороший сегодня всё-таки день)))снежок такой классный на улице!)',
 '@winteat спасибо и тебе приятного денечка, хоть погода и не айс :) @HelenaBrts @akutata11 @ira_dudchik @nalbakrinova @Pravodumru @YDubos']

In [41]:
TEXT = "весна наступила"

In [42]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['RT @AnastasiaDrobna: Осень наступила вовремя, зима тоже по расписанию, я надеюсь,что весна тоже будет пунктуальной !!!!! )))))))))))))',
 'ну вот и наступил этот день:) #совершеннолетие http://t.co/Gr7iCjrBN6',
 'Красота!)“@somsok5412: Зимний лес. http://t.co/76pkvWJrpJ”',
 'RT @DianaTelia: Зимний Лес))) http://t.co/KRmP6lddy7',
 'Было такое,буквально весной:) я-дура http://t.co/pC8i4HrESr']

In [43]:
get_response(TEXT, ft_index, modelFT, index_map)

['Люблю, кода идет снег)). Может он предаст нам новогоднее настроение))',
 '@Lira_Belaqua прекрасная прогулка получится. Я люблю гулять, когда только снег выпал )',
 'наш город всегда стоит) Погода лишь немного усугубляет этот процесс',
 'да) теперь нужно дождаться дождливой погоды для джинс, и можно красить) #TeamFollowWack',
 '@loveKorea96  в зимней курточке и шапочке зиму не так заметно,попробуй побегать раздетой,сейчас же почувствуешь)']

In [44]:
TEXT = "подарили цветы"

In [45]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Подарил ведь год назад\nВспомнилось что-то:DD http://t.co/HFd3dNergz',
 'RT @44_bendyuzhenko: Даня подарил мне кепочку^^\nНяняня;)',
 'Братик подарил вот такой вот цветоооочище) http://t.co/zE7o1aef55',
 'кто мне подарит такого мишку?) http://t.co/gc9FjdKKgK',
 'RT @skittles_skittl: Надо бы подарить слуховой аппарат моему пню @grigoraash :**']

In [46]:
get_response(TEXT, ft_index, modelFT, index_map)

['RT @svdolgoff: @Spartak83 @SSobolini Тебе рогатку подарить ? ))))',
 'Скинтесь и подарите мне мотик буду рада вам всем кому не жалко подарить хоть копеечку)))',
 'RT @stadnik_dr: А МНЕ НЮТА ПОДАРОК ПОДАРИЛА\nЯ КРУТЫШКА:3\nСПАСИБО,МИЛАЯ:*',
 '-Спасибо тебе за палочку-выручалочку))\n-ты что,подарила ей фаллоиммитатор?кхкхкхкх',
 'RT @44_bendyuzhenko: Даня подарил мне кепочку^^\nНяняня;)']

Обе модели справились с задачей хорошо, выдались релевантные твиты.